In [232]:
import pandas as pd
import numpy as np
from docplex.mp.environment import Environment

In [233]:
distance_matrix=np.load("/Users/portia_murray/Dropbox/PhD/Python_scripts/VersusVirus/ch_vs_virus_416/data/hospital_resources/dist_mat.npy")

In [229]:
Hospital_data=pd.read_csv(r'/Users/portia_murray/Dropbox/PhD/Python_scripts/VersusVirus/ch_vs_virus_416/data/hospital_resources/Hospital_resources_portia.csv',encoding="ISO-8859-1")
hospitals=range(0,len(Hospital_data['Inst']))
#Please note that AI, AR, and OW have been removed because they have no acute cases and have no intensive care facilities. If acute cases arise, they will have to be allocated other Kantons
kantons=['AG','BE','BL','BS','FR','GE','GL','GR','JU','LU','NE','NW','SG','SH','SO','SZ','TG','TI','UR','VD','VS','ZG','ZH']
VentDemand=pd.read_csv(r'/Users/portia_murray/Dropbox/PhD/Python_scripts/VersusVirus/ch_vs_virus_416/data/Patient_data/covid19_vent_switzerland_openzh.csv')
Hospitalized=pd.read_csv(r'/Users/portia_murray/Dropbox/PhD/Python_scripts/VersusVirus/ch_vs_virus_416/data/Patient_data/covid19_hospitalized_switzerland_openzh.csv')
horizon=range(0,39)
Hospitalized=Hospitalized.fillna(0)[0:39]
VentDemand=VentDemand.fillna(0)[0:39]
ventdemand=VentDemand.loc[38,kantons].values

In [143]:
Hospital_data

,Unnamed: 0,KT,Inst,PersA,PersP,BettenStatA,pBettenBelStatA,ICU_beds,Ventilators,pICUinKanton
0,0,BS,UniversitÌÎ_tsspital Basel,913.50,1595.22,633.000000,92.463806,38,28,NaN
1,1,GE,Les HÌÎåÇpitaux Universitaires de GenÌÎåÂve HUG,1829.32,4498.96,1054.282192,85.289218,63,47,NaN
2,2,VD,CHUV Centre Hospitalier Universitaire Vaudois,1653.73,3400.37,929.000000,82.588437,56,41,NaN
3,3,ZH,UniversitÌÎ_tsspital ZÌÎ_rich,1221.75,2574.21,953.884931,78.680694,57,42,NaN
4,4,BE,Insel Gruppe AG (universitÌÎ_r),1355.66,2739.25,876.298630,83.329008,53,39,NaN
...,...,...,...,...,...,...,...,...,...,...
62,230,GE,Clinique de Carouge,14.78,40.46,15.000000,28.566210,1,1,NaN
63,257,ZH,UniversitÌÎ_ts-Kinderspital ZÌÎ_rich das Spita...,302.29,683.98,167.128767,75.649979,10,7,NaN
64,258,SG,Ostschweizer Kinderspital,85.25,272.69,74.000000,81.728989,4,3,NaN
65,259,BS,UniversitÌÎ_ts-Kinderspital beider Basel (UKBB),157.55,411.82,118.501370,72.873558,7,5,NaN


In [230]:
hospitals_per_kanton={}
for kanton in kantons:
    tempHosp=Hospital_data[Hospital_data['KT']==kanton]
    hospitals_per_kanton[kanton]=tempHosp['ICU_beds'].sum()
for i in range(0,len(Hospital_data)):
    Hospital_data.loc[i,'pICUinKanton']=Hospital_data.loc[i,'ICU_beds']/hospitals_per_kanton[Hospital_data.loc[i,'KT']]

In [231]:
HVentDemand=pd.DataFrame(index=range(0,39),columns=hospitals)
for h in hospitals:
    for t in range(0,39):
        HVentDemand.loc[t,h]=round(Hospital_data.loc[h,'pICUinKanton']*VentDemand.loc[t,Hospital_data.loc[h,'KT']])
HVentDemand.loc[0,:].sum()

0.0

In [164]:
HVentDemand.sum().sum()

2544.0

In [165]:
VentDemand[kantons].sum().sum()

2548.0

In [169]:
Hospital_data

,Unnamed: 0,KT,Inst,PersA,PersP,BettenStatA,pBettenBelStatA,ICU_beds,Ventilators,pICUinKanton
0,0,BS,UniversitÌÎ_tsspital Basel,913.50,1595.22,633.000000,92.463806,38,28,0.644068
1,1,GE,Les HÌÎåÇpitaux Universitaires de GenÌÎåÂve HUG,1829.32,4498.96,1054.282192,85.289218,63,47,0.875000
2,2,VD,CHUV Centre Hospitalier Universitaire Vaudois,1653.73,3400.37,929.000000,82.588437,56,41,0.491228
3,3,ZH,UniversitÌÎ_tsspital ZÌÎ_rich,1221.75,2574.21,953.884931,78.680694,57,42,0.243590
4,4,BE,Insel Gruppe AG (universitÌÎ_r),1355.66,2739.25,876.298630,83.329008,53,39,0.321212
...,...,...,...,...,...,...,...,...,...,...
62,230,GE,Clinique de Carouge,14.78,40.46,15.000000,28.566210,1,1,0.013889
63,257,ZH,UniversitÌÎ_ts-Kinderspital ZÌÎ_rich das Spita...,302.29,683.98,167.128767,75.649979,10,7,0.042735
64,258,SG,Ostschweizer Kinderspital,85.25,272.69,74.000000,81.728989,4,3,0.053333
65,259,BS,UniversitÌÎ_ts-Kinderspital beider Basel (UKBB),157.55,411.82,118.501370,72.873558,7,5,0.118644


In [237]:
from docplex.mp.model import Model
mdl = Model(name="Ventilators")
# --- decision variables ---
mdl.stocked_ventilators = mdl.integer_var_matrix(horizon,hospitals,name='stocked_ventilators')
mdl.yv_transport = mdl.binary_var_cube(horizon,hospitals,hospitals, name='yv_transport')
mdl.num_transport = mdl.integer_var_cube(horizon,hospitals,hospitals, name='num_transport')
mdl.dummy_transport=mdl.integer_var_cube(horizon,hospitals,hospitals,name='dummy_transport')
mdl.slack = mdl.integer_var_matrix(horizon,hospitals, name='slack')

In [258]:
# --- constraints ---
#Initiate original stock
for t in horizon:
    if t==0:
        mdl.add_constraints(mdl.stocked_ventilators[t,i]==Hospital_data.loc[i,'Ventilators'] for i in hospitals)
    else:
        mdl.add_constraints(mdl.stocked_ventilators[t,i]==mdl.stocked_ventilators[t-1,i]+mdl.sum(mdl.dummy_transport[t,j,i] for j in hospitals)-mdl.sum(mdl.dummy_transport[t,i,j] for j in hospitals) for i in hospitals)
# dummy variable
mdl.add_constraints(mdl.dummy_transport[t,i,j]<=mdl.yv_transport[t,i,j]*Hospital_data.loc[i,'Ventilators'] for t in horizon for i in hospitals for j in hospitals)
mdl.add_constraints(mdl.dummy_transport[t,i,j]<=mdl.num_transport[t,i,j] for i in hospitals for j in hospitals)
mdl.add_constraints(mdl.dummy_transport[t,i,j]>=mdl.num_transport[t,i,j]-Hospital_data.loc[i,'Ventilators']*(1-mdl.yv_transport[t,i,j]) for t in horizon for i in hospitals for j in hospitals)
# cannot transport to/from same hospital
mdl.add_constraints(mdl.yv_transport[t,i,j]==0 for t in horizon for i in hospitals for j in hospitals if i==j)
mdl.add_constraints(mdl.num_transport[t,i,j]<=mdl.yv_transport[t,i,j]*Hospital_data.loc[i,'Ventilators'] for t in horizon for i in hospitals for j in hospitals)
# Number of patients on ventilators + slack (only to be used in case there isn't enough ventilators in the country and is to be minimized in objective) 
# must be less than or equal to the ventilations currently in hospital + ventilators arriving hospital - ventilators leaving the hospital
mdl.add_constraints(HVentDemand.loc[t,i]*2+mdl.slack[t,i]<=mdl.stocked_ventilators[t,i]-mdl.sum(mdl.dummy_transport[t,i,j] for j in hospitals)+mdl.sum(mdl.dummy_transport[t,j,i] for t in horizon for j in hospitals) for i in hospitals)

# total number of transported ventilators leaving hospital must be less than or equal to the total number of ventilators ---
mdl.add_constraints(mdl.sum(mdl.num_transport[t,i,j]for j in hospitals)<=mdl.stocked_ventilators[t,i] for t in horizon for i in hospitals)

# --- objective function --- minimise total distance travelled to transport + slack variable in case there are not enough variables
mdl.minimize(mdl.sum(mdl.yv_transport[t,i,j]*distance_matrix[i,j] for t in horizon for i in hospitals for j in hospitals)+mdl.sum(mdl.slack[t,i]**2 for t in horizon for i in hospitals))


In [249]:
HVentDemand.loc[38,:].sum()*3

792.0

In [198]:
Allocation=pd.DataFrame(index=hospitals,columns=hospitals)
for i in hospitals:
    for j in hospitals:
        Allocation.loc[]

847

In [ ]:
assert mdl.solve(), "!!! Solve of the model fails"
mdl.report()

In [ ]:
for i in hospitals:
    print(mdl.stocked_ventilators[0,i].solution_value)

In [ ]:
for i in hospitals:
    for j in hospitals:
        print(mdl.slack[t,i].solution_value)